In [20]:
import pandas as pd
import os

# Define the folder path where your CSV files are located
folder_path = 'Samples for WSP/SM/'

# Initialize an empty list to store the DataFrames
dataframes = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all the DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Create the output folder if it doesn't exist
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)

# Print the combined DataFrame
combined_df['Date'].unique()
combined_df


,Date,Site ID,Cycle sequence,Cycle start time,Planned cycle length,Actual cycle length,Required cycle length,DS,Strategic approach controlling cycle time,Subsystem number,Stretched phase,A,B,C,D,E,F,G
0,8/03/2023,3072,1,00:00:00,50,50,50,11,337,64,A,180.0,1.0,1.0,16.0,16.0,16.0,NaN
1,8/03/2023,3035,1,00:00:00,60,60,60,15,124,30,A,159.0,1.0,3.0,29.0,25.0,7.0,5.0
2,8/03/2023,2378,1,00:00:00,60,60,60,9,14,5,A,178.0,12.0,21.0,5.0,12.0,NaN,NaN
3,8/03/2023,3099,1,00:00:00,60,60,60,0,0,70,B,13.0,173.0,1.0,14.0,15.0,13.0,NaN
4,8/03/2023,3083,1,00:00:00,60,60,60,0,0,83,A,165.0,30.0,100.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383386,10/03/2023,3084,1285,23:59:00,60,60,60,0,0,77,A,168.0,1.0,1.0,20.0,20.0,1.0,20.0
383387,10/03/2023,3012,1127,23:59:00,65,65,65,15,93,21,A,192.0,1.0,17.0,19.0,NaN,NaN,NaN
383388,10/03/2023,3085,1439,23:59:00,60,60,60,0,0,79,A,159.0,20.0,12.0,102.0,NaN,NaN,NaN
383389,10/03/2023,2370,2274,23:59:00,50,50,50,4,54,8,A,178.0,1.0,1.0,14.0,17.0,19.0,NaN


In [22]:
# Iterate over each unique date in the "Date" column
for date in combined_df['Date'].unique():
    # Replace "/" with "-" in the date
    formatted_date = date.replace("/", "-")
    
    # Create a folder for the current date
    date_folder = os.path.join(output_folder, formatted_date)
    os.makedirs(date_folder, exist_ok=True)
    single_date_df = combined_df[combined_df['Date'] == date]

    # Convert 'Cycle_starttime' and 'Date' columns to datetime format
    single_date_df['Cycle start time'] = pd.to_datetime(single_date_df['Cycle start time'])

    single_date_df['Date'] = pd.to_datetime(single_date_df['Date'])

    # Round the 'Cycle_starttime' column to the nearest 5-minute interval
    single_date_df['Rounded_Time'] = single_date_df['Cycle start time'].dt.floor('5T')

    # Combine 'Date' and 'Rounded_Time' into a new column
    single_date_df['Combined_DateTime'] = single_date_df['Date'].astype(str) + ' ' + single_date_df['Rounded_Time'].dt.time.astype(str)

    # Rename the 'Rounded_Combined_DateTime' column to 'Combined_DateTime'
    single_date_df = single_date_df.rename(columns={'Rounded_Combined_DateTime': 'Combined_DateTime'})
    # Move the 'Combined_DateTime' column to be the 5th column
    cols = list(single_date_df.columns)
    cols.insert(4, cols.pop(cols.index('Combined_DateTime')))
    single_date_df = single_date_df[cols]
    
    for site_id in single_date_df['Site ID'].unique():
        site_id_df = single_date_df[single_date_df['Site ID'] == site_id]
        # Remove duplicate values in the 'Combined_DateTime' column
        site_id_df.drop_duplicates(subset='Combined_DateTime', inplace=True)
        # Drop Cycle_starttime column
        site_id_df = site_id_df.drop('Cycle start time', axis=1)
        single_date_df_folder = output_folder+"/"+formatted_date+f"/{site_id}.csv"
        # Write the DataFrame to a CSV file
        site_id_df.to_csv(single_date_df_folder, index=False)

single_date_df

C:\Users\nzak30381\AppData\Local\Temp\ipykernel_26964\731247464.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_date_df['Cycle start time'] = pd.to_datetime(single_date_df['Cycle start time'])
C:\Users\nzak30381\AppData\Local\Temp\ipykernel_26964\731247464.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_date_df['Date'] = pd.to_datetime(single_date_df['Date'])
C:\Users\nzak30381\AppData\Local\Temp\ipykernel_26964\731247464.py:17: SettingWithCopyWarning: 
A value is trying to be set on

,Date,Site ID,Cycle sequence,Cycle start time,Combined_DateTime,Planned cycle length,Actual cycle length,Required cycle length,DS,Strategic approach controlling cycle time,Subsystem number,Stretched phase,A,B,C,D,E,F,G,Rounded_Time
255600,2023-10-03,2366,1,2023-06-21 00:00:00,2023-10-03 00:00:00,60,60,60,9,2,1,A,187.0,17.0,12.0,12.0,NaN,NaN,NaN,2023-06-21 00:00:00
255601,2023-10-03,3032,1,2023-06-21 00:00:00,2023-10-03 00:00:00,60,60,60,0,0,52,A,143.0,1.0,17.0,112.0,15.0,NaN,NaN,2023-06-21 00:00:00
255602,2023-10-03,3099,1,2023-06-21 00:00:00,2023-10-03 00:00:00,60,60,60,0,0,70,B,15.0,173.0,1.0,16.0,11.0,13.0,NaN,2023-06-21 00:00:00
255603,2023-10-03,3083,1,2023-06-21 00:00:00,2023-10-03 00:00:00,60,60,60,6,445,83,A,166.0,29.0,100.0,NaN,NaN,NaN,NaN,2023-06-21 00:00:00
255604,2023-10-03,3111,1,2023-06-21 00:00:00,2023-10-03 00:00:00,50,50,50,42,504,90,A,168.0,1.0,1.0,16.0,24.0,20.0,NaN,2023-06-21 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383386,2023-10-03,3084,1285,2023-06-21 23:59:00,2023-10-03 23:55:00,60,60,60,0,0,77,A,168.0,1.0,1.0,20.0,20.0,1.0,20.0,2023-06-21 23:55:00
383387,2023-10-03,3012,1127,2023-06-21 23:59:00,2023-10-03 23:55:00,65,65,65,15,93,21,A,192.0,1.0,17.0,19.0,NaN,NaN,NaN,2023-06-21 23:55:00
383388,2023-10-03,3085,1439,2023-06-21 23:59:00,2023-10-03 23:55:00,60,60,60,0,0,79,A,159.0,20.0,12.0,102.0,NaN,NaN,NaN,2023-06-21 23:55:00
383389,2023-10-03,2370,2274,2023-06-21 23:59:00,2023-10-03 23:55:00,50,50,50,4,54,8,A,178.0,1.0,1.0,14.0,17.0,19.0,NaN,2023-06-21 23:55:00
